# Orangutan Organoid Scanpy

~15000 Cells from a 10x run 

In [3]:
%matplotlib inline

import matplotlib
#matplotlib.use('agg') # plotting backend compatible with screen
import scanpy
import scanpy.api as sc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import logging as logg
import os
import loompy
import scipy
import re
import anndata



In [4]:
sc.settings.verbosity = 2
sc.settings.autosave = True # save figures, do not show them
sc.settings.set_figure_params(dpi=300) # set sufficiently high resolution for saving

In [5]:
inputfile = os.path.expanduser('/ye/yelabstore2/mtschmitz/seq/AlignedOrangutanOrganoid/Exonic/orangutanorganoid_Out/outs')
# Run louvain clustering on true gene expression values
velocityFile = os.path.expanduser('/ye/yelabstore2/mtschmitz/seq/AlignedOrangutanOrganoid/Exonic/orangutanorganoid_Out/velocyto/orangutanorganoid_Out.loom')

In [6]:
adata = sc.read_10x_h5('/home/mt/code/data/AlignedOrangutanOrganoid/Exonic/orangutanorganoid_Out/outs/filtered_gene_bc_matrices_h5.h5','refdata-celranger-Pabe2-toplevel')

reading /home/mt/code/data/AlignedOrangutanOrganoid/Exonic/orangutanorganoid_Out/outs/filtered_gene_bc_matrices_h5.h5 Variable names are not unique. To make them unique, call `.var_names_make_unique`.
(0:00:00.33)


In [2]:
def checkDuplicates(a):
    return(len(a) == len(set(a)))
    
def gsub(regex, sub, l):
    return([re.sub(regex, sub, x) for x in l])

def orderIntersectLists(a,b):
    set(a).intersection(b)

def addCleanObsNames(adata,regex="-[0-9]"):
    try:
        adata.obs.loc[:,"clean_obs_names"]=adata.obs.loc[:,"clean_obs_names"]
    except:
        adata.obs.loc[:,"clean_obs_names"]=adata.obs_names
    adata.obs.loc[:,"clean_obs_names"]=gsub(regex, "",adata.obs.loc[:,"clean_obs_names"])
    return(adata)
    
def removeVelocyto(adata):
    adata.U=None
    adata.S=None
    return(adata)
    
def match(a,b):
    return([ b.index(x) if x in b else None for x in a ])
    
    
    
def addVelocyto(adata,loomfile):
    if not hasattr(adata, 'U'):
        print('adding velocyto')
        ds=loompy.connect(loomfile)
        row_attrs = pd.DataFrame.from_dict(dict(ds.row_attrs.items()))
        col_attrs = pd.DataFrame.from_dict(dict(ds.col_attrs.items()))

        col_attrs.loc[:,"clean_obs_names"] = cell_names = gsub("^[a-zA-Z0-9_]+:", "",col_attrs.loc[:,'CellID'])
        col_attrs.loc[:,"clean_obs_names"] = cell_names = gsub("x","",col_attrs.loc[:,'clean_obs_names'])
        
        cell_names=list(adata.obs.loc[:,'clean_obs_names'])
        cell_names = [cell for cell in list(col_attrs.loc[:,'clean_obs_names']) if cell in cell_names]
        cell_index = match(list(col_attrs.loc[:,'clean_obs_names']),list(adata.obs.loc[:,'clean_obs_names']))
        
        gene_names = [gene for gene in row_attrs.loc[:,'Accession'] if gene in adata.var.loc[:,'gene_ids']]
        gene_index = match(list(row_attrs.loc[:,'Accession']),list(adata.var.loc[:,'gene_ids']))
        #print(adata.var)
        #print(row_attrs)
        #print(match(list(row_attrs.loc[:,'Accession']),list(adata.var.loc[:,'gene_ids'])))
        #print(gene_index)
        
        #cols_to_use = col_attrs.columns.difference( adata.obs.columns)
        adata.obs = pd.merge( adata.obs,col_attrs,how='outer',on="clean_obs_names")
        adata.obs.set_index('clean_obs_names')

        #cols_to_use = row_attrs.columns.difference(adata.vars.columns)
        adata.var=pd.merge( adata.var,row_attrs,how='outer',right_on="Accession",left_on="gene_ids")
        adata.var.set_index('Gene')
        
        if len(cell_names) == 0:
            raise ValueError(
                'Cell names in loom file do not match cell names in AnnData.')
        
        from anndata.base import _normalize_index
        norm_gene_index=_normalize_index(gene_index, pd.RangeIndex(len(gene_index)))
        norm_cell_index=_normalize_index(cell_index, pd.RangeIndex(len(cell_index)))
        
        # subset to cells and genes present in adata. Do this with lists of integer indices
        #adata.S = anndata.AnnData(ds.layer['spliced'].sparse().tocsr().T)
        #adata.U = anndata.AnnData(ds.layer['unspliced'].sparse().tocsr().T)
        adata.S = anndata.AnnData(ds.layer['spliced'].sparse(norm_gene_index,norm_cell_index).tocsr().T)
        adata.U = anndata.AnnData(ds.layer['unspliced'].sparse(norm_gene_index,norm_cell_index).tocsr().T)
        
        ds.close()
    return(adata,adataS,adataU)

def openVelocyto(adata,loomfile):
    print('adding velocyto')
    ds=loompy.connect(loomfile)
    row_attrs = pd.DataFrame.from_dict(dict(ds.row_attrs.items()))
    col_attrs = pd.DataFrame.from_dict(dict(ds.col_attrs.items()))

    col_attrs.loc[:,"clean_obs_names"] = cell_names = gsub("^[a-zA-Z0-9_]+:", "",col_attrs.loc[:,'CellID'])
    col_attrs.loc[:,"clean_obs_names"] = cell_names = gsub("x","",col_attrs.loc[:,'clean_obs_names'])

    cell_names=list(adata.obs.loc[:,'clean_obs_names'])
    cell_names = [cell for cell in list(col_attrs.loc[:,'clean_obs_names']) if cell in cell_names]
    cell_index = match(list(col_attrs.loc[:,'clean_obs_names']),list(adata.obs.loc[:,'clean_obs_names']))

    gene_names = [gene for gene in row_attrs.loc[:,'Accession'] if gene in adata.var.loc[:,'gene_ids']]
    gene_index = match(list(row_attrs.loc[:,'Accession']),list(adata.var.loc[:,'gene_ids']))

    #cols_to_use = col_attrs.columns.difference( adata.obs.columns)
    adata.obs = pd.merge( adata.obs,col_attrs,how='left',on="clean_obs_names")
    adata.obs.set_index('clean_obs_names',drop=False)

    #cols_to_use = row_attrs.columns.difference(adata.vars.columns)
    vardata=pd.merge( adata.var,row_attrs,how='inner',right_on="Accession",left_on="gene_ids")
    adata.var=vardata
    adata.var.set_index('Gene',drop=False)

    if len(cell_names) == 0:
        raise ValueError(
            'Cell names in loom file do not match cell names in AnnData.')

    from anndata.base import _normalize_index
    gene_index=[x for x in gene_index if x is not None]
    cell_index=[x for x in cell_index if x is not None]
    norm_gene_index=_normalize_index(gene_index, pd.RangeIndex(len(gene_index)))
    norm_cell_index=_normalize_index(cell_index, pd.RangeIndex(len(cell_index)))

    #adata.S = anndata.AnnData(ds.layer['spliced'].sparse().tocsr().T)
    #adata.U = anndata.AnnData(ds.layer['unspliced'].sparse().tocsr().T)
    # subset to cells and genes present in adata. Do this with lists of integer indices
    adataS = anndata.AnnData(ds.layer['spliced'].sparse(norm_gene_index,norm_cell_index).tocsr().T)
    adataU = anndata.AnnData(ds.layer['unspliced'].sparse(norm_gene_index,norm_cell_index).tocsr().T)
    adataS.obs=pd.merge( adata.obs,col_attrs,how='inner',on="clean_obs_names")
    adataU.obs=pd.merge( adata.obs,col_attrs,how='inner',on="clean_obs_names")
    adataS.obs.set_index('clean_obs_names', drop=False)
    adataU.obs.set_index('clean_obs_names', drop=False)
    print(vardata.shape)
    print(adataS.shape)
    adataS.var=vardata
    adataU.var=vardata
    adataS.var.set_index('Gene', drop=False)
    adataU.var.set_index('Gene', drop=False)
    adataU.var_names=adataU.var.loc[:,"Gene"]
    adataS.var_names=adataS.var.loc[:,"Gene"]

    ds.close()
    return(adata,adataS,adataU)



In [ ]:
loomfile= os.path.expanduser('~/code/data/AlignedOrangutanOrganoid/Exonic/orangutanorganoid_Out/velocyto/orangutanorganoid_Out.loom')
ds=loompy.connect(loomfile)
row_attrs = pd.DataFrame.from_dict(dict(ds.row_attrs.items()))
col_attrs = pd.DataFrame.from_dict(dict(ds.col_attrs.items()))

col_attrs.loc[:,"clean_obs_names"] = cell_names = gsub("^[a-zA-Z0-9_]+:", "",col_attrs.loc[:,'CellID'])
col_attrs.loc[:,"clean_obs_names"] = cell_names = gsub("x","",col_attrs.loc[:,'clean_obs_names'])
cell_names=list(adata.obs.loc[:,'clean_obs_names'])
cell_names = [cell for cell in list(col_attrs.loc[:,'clean_obs_names']) if cell in cell_names]
cell_index = match(list(col_attrs.loc[:,'clean_obs_names']),list(adata.obs.loc[:,'clean_obs_names']))
gene_names = [gene for gene in row_attrs.loc[:,'Accession'] if gene in adata.var.loc[:,'gene_ids']]
gene_index = match(list(row_attrs.loc[:,'Accession']),list(adata.var.loc[:,'gene_ids']))



In [7]:
adata=addCleanObsNames(adata)
adata,adataS,adataU=openVelocyto(adata, os.path.expanduser('~/code/data/AlignedOrangutanOrganoid/Exonic/orangutanorganoid_Out/velocyto/orangutanorganoid_Out.loom'))

adding velocyto
(26224, 7)
(14742, 26224)
Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.


In [ ]:
#adata.S = anndata.AnnData(ds.layer['spliced'].sparse(pd.Series(gene_index),pd.Series(cell_index)).tocsr().T)
#adata.S = anndata.AnnData(ds.layer['spliced'].sparse(pd.Index(gene_index),pd.Index(cell_index)).tocsr().T)
#adata.S = anndata.AnnData(ds.layer['spliced'].sparse(tuple(gene_index),tuple(cell_index)).tocsr().T)
#print(anndata.base._normalize_index(cell_index, pd.RangeIndex(len(cell_index))))
#adata.S = anndata.AnnData(ds.layer['spliced'].sparse(pd.RangeIndex(len(gene_index)),pd.RangeIndex(len(cell_index))).tocsr().T)
from anndata.base import _normalize_index
norm_gene_index=_normalize_index(gene_index, pd.RangeIndex(len(gene_index)))
norm_cell_index=_normalize_index(cell_index, pd.RangeIndex(len(cell_index)))
print(norm_gene_index[0:10])
print(gene_index[0:10])
print(row_attrs.loc[0:10,'Accession'])
print(adata.var.loc[0:10,'gene_ids'])
#adata.S = anndata.AnnData(ds.layer['spliced'].sparse(norm_gene_index,norm_cell_index).tocsr().T)


In [13]:
print(adataS)

AnnData object with n_obs × n_vars = 14742 × 26224 
    obs: 'clean_obs_names', 'CellID_x', 'Clusters_x', '_X_x', '_Y_x', 'CellID_y', 'Clusters_y', '_X_y', '_Y_y'
    var: 'gene_ids', 'Accession', 'Chromosome', 'End', 'Gene', 'Start', 'Strand'


In [ ]:
#cols_to_use = col_attrs.columns.difference( adata.obs.columns)
adata.obs = pd.merge( adata.obs,col_attrs,how='outer',on="clean_obs_names")
adata.obs.set_index('clean_obs_names')

#cols_to_use = row_attrs.columns.difference(adata.vars.columns)
adata.var=pd.merge( adata.var,row_attrs,how='outer',right_on="Accession",left_on="gene_ids")
adata.var.set_index('Gene')

adata.S = anndata.AnnData(ds.layer['spliced'].sparse().tocsr().T)



In [ ]:
#adata=addCleanObsNames(adata)
#adata=addVelocyto(adata, os.path.expanduser('~/code/data/AlignedOrangutanOrganoid/Exonic/orangutanorganoid_Out/velocyto/orangutanorganoid_Out.loom'))

In [ ]:
def filterSplicedAndUnspliced(adata,min_cells):
    print("todo")
    
def runVelocity(adata,ad_s,ad_u):
    subset, _ = sc.pp.filter_genes(ad_u.X, min_cells=50)
    print(np.sum(subset))
    ad_s = ad_s[:, subset]
    ad_u = ad_u[:, subset]
    #ad_s.var_names = np.array(ad_u.var.loc[:,"Gene"])[subset]

    # loop over genes
    from scipy.sparse import dok_matrix
    offset = np.zeros(ad_s.shape[1], dtype='float32')
    gamma = np.zeros(ad_s.shape[1], dtype='float32')
    X_du = dok_matrix(ad_s.shape, dtype='float32')
    for i in range(ad_s.shape[1]):
        x = ad_s.X[:, i].toarray()
        y = ad_u.X[:, i].toarray()
        subset = np.logical_and(x > 0, y > 0)
        x = x[subset]
        y = y[subset]
        X = np.c_[np.ones(len(x)), x]
        offset[i], gamma[i] = np.linalg.pinv(X).dot(y)
        subset_indices = np.flatnonzero(subset)
        index = subset_indices, np.array([i for dummy in subset_indices])
        X_du[index] = y - gamma[i]*x - offset[i]
        #plt.scatter(x, y)
        #plt.scatter(x, gamma[i]*x + offset[i])
        #plt.scatter(x, X_du[index].toarray()[0])
        #plt.show()
    X_du = X_du.tocoo().tocsr()

    #Need to run this outside
    #sc.pp.neighbors(adata[:,subset], n_neighbors=100)
    #Also moved this outside
    #graph = compute_velocity_graph(adata, ad_u, X_du)
    return(adata,ad_u,X_du)

def compute_velocity_graph(adata, adata_u, X_du):
    if (adata.shape[0] != adata_u.shape[0]
        or adata_u.shape[0] != X_du.shape[0]
        or X_du.shape[0] != adata.shape[0]):
        raise ValueError('Number of cells do not match.')

    from scanpy.neighbors import Neighbors, get_indices_distances_from_sparse_matrix
    neigh = Neighbors(adata)
    knn_indices, knn_distances = get_indices_distances_from_sparse_matrix(
        neigh.distances, neigh.n_neighbors)
    n_obs = adata.n_obs
    n_neighbors = neigh.n_neighbors

    from numpy.linalg import norm
    X_u = adata_u.X.toarray()
    X_du = X_du.astype('float32').toarray()

    def fill_graph():
        rows = np.zeros((n_obs * n_neighbors), dtype=np.int64)
        cols = np.zeros((n_obs * n_neighbors), dtype=np.int64)
        vals = np.zeros((n_obs * n_neighbors), dtype=np.float32)
        for i in range(n_obs):
            if i % 100 == 0:
                try:
                    logg.info('{}/{},'.format(i, n_obs))
                    print(i,n_obs)
                except:
                    print(i,n_obs)
            for nj in range(n_neighbors):
                j = knn_indices[i, nj]
                if j != i:
                    du_i = X_du[i]
                    du_ji = X_u[j] - X_u[i]
                    subset = np.logical_or(du_ji != 0, du_i != 0)
                    du_i = du_i[subset]
                    du_ji = du_ji[subset]
                    # dividing this by norm(du_i) doesn't make much of a difference
                    val = np.dot(du_ji, du_i) / norm(du_ji) / norm(du_i)
                    # if val > 0, this means transitioning from i to j,
                    # convention of standard stochastic matrices even though
                    # this isn't one
                    # the problem with velocities at the boundaries of the knn
                    # graph is that, no matter in which direction they point,
                    # they anticorrelate with all neighbors: hence, one will
                    # always observe "out-going" velocity even if there is no
                    # indication for that
                    if True:  # val > 0:
                        rows[i * n_neighbors + nj] = j
                        cols[i * n_neighbors + nj] = i
                        vals[i * n_neighbors + nj] = val
        return rows, cols, vals

    rows, cols, vals = fill_graph()
    from scipy.sparse import coo_matrix
    graph = coo_matrix((vals, (rows, cols)), shape=(n_obs, n_obs))
    graph.eliminate_zeros()
    
    return(graph.tocsr())


def compute_arrows_embedding(adata,basis="tsne"):
    if 'graph' not in adata.uns:
        raise ValueError('`arrows=True` requires `tl.rna_velocity` to be run before.')
    adjacency = adata.uns['graph']
    # loop over columns of adjacency, this is where transitions start from
    from numpy.linalg import norm
    V = np.zeros((adjacency.shape[0],  adata.obsm['X_' + basis].shape[1]), dtype='float32')
    for i, n in enumerate(adjacency.T):  # loop over columns (note the transposition)
        for j in n.nonzero()[1]:  # these are row indices
            diff = adata.obsm['X_' + basis][j] - adata.obsm['X_' + basis][i]
            # need the normalized difference vector: the distance in the embedding
            # might be completely meaningless
            diff /= norm(diff)
            V[i] += adjacency[j, i] * diff
    logg.info('added \'V_{}\' to `.obsm`'.format(basis))
    adata.obsm['V_' + basis] = V
    X = adata.obsm['X_' + basis]
    for ax in axs:
        quiver_kwds = arrows_kwds if arrows_kwds is not None else {}
        ax.quiver(X[:, 0], X[:, 1], V[:, 0], V[:, 1], **quiver_kwds)




In [ ]:
x=[1,3,2,5]
y=[0,0,10,0]
X=np.c_[[1,1,1,1],x]
print(X)
print(np.linalg.pinv(X))
print(np.linalg.pinv(X).dot(y))
adata.obsm['X_pca']

In [ ]:
#subset, _ = sc.pp.filter_genes(adata.U.X, min_cells=50)
#print(np.sum(adata.X,axis=0))
#print(np.sum(adata.U.X,axis=0))
#print(np.sum(adata.S.X,axis=0))
#print([np.sum(adata.U.X,axis=0).tolist()[0],np.sum(adata.S.X,axis=0).tolist()[0]])
#plt.scatter(np.sum(adata.U.X,axis=0).tolist()[0],np.sum(adata.S.X,axis=0).tolist()[0])
#plt.scatter(adata.U.X.toarray()[1:100,1:1000],adata.S.X.toarray()[1:100,1:1000])
#plt.scatter(adata.X.toarray()[1:100,1:1000],adata.S.X.toarray()[1:100,1:1000])
#plt.show()
cors=[]
x=adata.X.toarray()
y=adataU.X.toarray()
for n in range(1000):
    #n = np.random.randint(10000)
    if np.sum(x[:,n])>0 and np.sum(y[:,n])>0:
        cors.append(scipy.stats.pearsonr(x[:,n], y[:,n]))
print(np.sum(x,axis=1)[1:100])
print(np.sum(y,axis=1)[1:100])

plt.hist([x[0] for x in cors])
plt.scatter(adata.X.toarray()[1:1000,1:100],adataS.X.toarray()[1:1000,1:100])
plt.show()

print(adataS.var.loc[1:10,:])
print(adata.var.loc[1:10,:])
print(adata.var_names)
adataU.raw

In [ ]:
#sc.pp.pca(adata)
#sc.pp.neighbors(adata,knn=100)
adata,ad_u,ad_X= runVelocity(adata,adataS,adataU)


In [ ]:
adata_u=ad_u
X_du=ad_X
if (adata.shape[0] != adata_u.shape[0]
    or adata_u.shape[0] != X_du.shape[0]
    or X_du.shape[0] != adata.shape[0]):
    raise ValueError('Number of cells do not match.')

from scanpy.neighbors import Neighbors, get_indices_distances_from_sparse_matrix
neigh = Neighbors(adata)
knn_indices, knn_distances = get_indices_distances_from_sparse_matrix(
    neigh.distances, neigh.n_neighbors)
n_obs = adata.n_obs
n_neighbors = neigh.n_neighbors

from numpy.linalg import norm
X_u = adata_u.X.toarray()
X_du = X_du.astype('float32').toarray()

def fill_graph():
    rows = np.zeros((n_obs * n_neighbors), dtype=np.int64)
    cols = np.zeros((n_obs * n_neighbors), dtype=np.int64)
    vals = np.zeros((n_obs * n_neighbors), dtype=np.float32)
    for i in range(n_obs):
        if i % 100 == 0:
            try:
                logg.info('{}/{},'.format(i, n_obs))
                print(i,n_obs)
            except:
                print(i,n_obs)
        for nj in range(n_neighbors):
            j = knn_indices[i, nj]
            if j != i:
                du_i = X_du[i]
                du_ji = X_u[j] - X_u[i]
                subset = np.logical_or(du_ji != 0, du_i != 0)
                du_i = du_i[subset]
                du_ji = du_ji[subset]
                # dividing this by norm(du_i) doesn't make much of a difference
                val = np.dot(du_ji, du_i) / norm(du_ji) / norm(du_i)
                # if val > 0, this means transitioning from i to j,
                # convention of standard stochastic matrices even though
                # this isn't one
                # the problem with velocities at the boundaries of the knn
                # graph is that, no matter in which direction they point,
                # they anticorrelate with all neighbors: hence, one will
                # always observe "out-going" velocity even if there is no
                # indication for that
                if True:  # val > 0:
                    rows[i * n_neighbors + nj] = j
                    cols[i * n_neighbors + nj] = i
                    vals[i * n_neighbors + nj] = val
    return rows, cols, vals

rows, cols, vals = fill_graph()
from scipy.sparse import coo_matrix
graph = coo_matrix((vals, (rows, cols)), shape=(n_obs, n_obs))
graph.eliminate_zeros()
graph.tocsr()


In [ ]:
adata.uns['graph']=compute_velocity_graph(adata,ad_u,ad_X)


In [ ]:
print(adata.uns['graph'])
compute_arrows_embedding(adata)

In [ ]:
#sc.tl.tsne(adata)
basis="pca"
if 'graph' not in adata.uns:
    raise ValueError('`arrows=True` requires `tl.rna_velocity` to be run before.')
adjacency = adata.uns['graph']
# loop over columns of adjacency, this is where transitions start from
from numpy.linalg import norm
V = np.zeros((adjacency.shape[0],  adata.obsm['X_' + basis].shape[1]), dtype='float32')
for i, n in enumerate(adjacency.T):  # loop over columns (note the transposition)
    for j in n.nonzero()[1]:  # these are row indices
        diff = adata.obsm['X_' + basis][j] - adata.obsm['X_' + basis][i]
        # need the normalized difference vector: the distance in the embedding
        # might be completely meaningless
        diff /= norm(diff)
        V[i] += adjacency[j, i] * diff
logg.info('added \'V_{}\' to `.obsm`'.format(basis))
adata.obsm['V_' + basis] = V
X = adata.obsm['X_' + basis]
#for ax in axs:
#    quiver_kwds = arrows_kwds if arrows_kwds is not None else {}
#    ax.quiver(X[:, 0], X[:, 1], V[:, 0], V[:, 1], **quiver_kwds)


In [ ]:
print(adata.obsm['V_tsne'])
print(adata.obs)
sc.pl.scatter(adata,x='_X',y='_Y',color='Clusters')
sc.tl.draw_graph(adata)
sc.pl.draw_graph(adata,arrows=True)